# **Advanced RAG**
This example constitutes a proof of concept in the llamaindex library to build a RAG pipeline that provides context to the language model and also identifies the documents used in the response.

## Load data

In [1]:
from llama_hub.file.pdf.base import PDFReader
from llama_index import SimpleDirectoryReader


documents = PDFReader().load_data(
    file="LLM_TRAIN/eBook-How-to-Build-a-Career-in-AI.pdf"
)

# alternative

# documents = SimpleDirectoryReader(
#     input_files=['LLM_TRAIN/eBook-How-to-Build-a-Career-in-AI.pdf']
# ).load_data()


len(documents)

41

## Parse Documents into Nodes using SentenceWindowRetrieval

In [2]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
# we can use sentence_splitter args to say how splits text into sentences

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    include_prev_next_rel=True,
    original_text_metadata_key="original_text",
)

## LLM and embeddings

In [1]:
from llama_index.embeddings import resolve_embed_model
#  BGE embedder from HuggingFace
# "local:BAAI/bge-small-en-v1.5"
embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")

# from llama_index import set_global_tokenizer
# # tokenizer for huggingface
# from transformers import AutoTokenizer

# set_global_tokenizer(
#     AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0").encode
# )

In [2]:
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate


llm = HuggingFaceLLM(
    # model_name="Deci/DeciLM-6b-instruct",
    # tokenizer_name="Deci/DeciLM-6b-instruct",

    # model_name="WeOpenML/Alpaca-7B-v1",  # alapca of stanford
    # tokenizer_name="WeOpenML/Alpaca-7B-v1",

    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    query_wrapper_prompt=PromptTemplate(
        "<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    # query_wrapper_prompt=PromptTemplate(template),

    context_window=2048,  # 4096
    max_new_tokens=256,  # 512
    model_kwargs={'trust_remote_code': True},
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    device_map="auto",
)

In [5]:
from llama_index import ServiceContext

# service context is a wrapper object that contains all the context needed for indexing
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

## Building the index

In [6]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    documents, service_context=sentence_context
)
# Save index to disk for later loading
sentence_index.storage_context.persist(persist_dir="LLM_TRAIN/sentence_index")

### Building the postprocessor
Optionality you can add a reranker

In [7]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

# This takes a value stored in the metadata and replaces a node text
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

## Query the index

In [8]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=3, node_postprocessors=[postproc]  # fetch the six most similarity
)

In [9]:
print(sentence_window_engine.query(
    "What are the keys to building a career in AI?"))

The keys to building a career in AI are:

1. Learning foundational technical skills - this includes topics such as programming, machine learning, and data analysis.
2. Finding a job - this involves pursuing projects, building a portfolio, and creating impact.
3. Finding a job in a specific field, such as AI engineering or data science.
4. Focusing on learning foundational skills over time to build a solid technical foundation.
5. Tackling large projects and working in teams to improve collaboration and communication skills.
6. Building a solid portfolio that demonstrates your skills and experience.
7. Building a network of industry professionals and potential employers.
8. Seeking out mentorship and certifications to improve your skills further.


## Obtain the documents from which the context is extracted

In [10]:
question = 'how can I start learning on the AI way?'
result = sentence_window_engine.query(question)
result.response  # get the response text

"The best way to start learning on the AI way is to start small and succeed. Instead of trying to exercise for 30 minutes a day, aim for doing just one push-up. This approach may be helpful for those who want to spend more time studying. Even if you learn nothing in that 10 seconds, you're building the habit of studying daily. On some days, you may end up studying for an hour or longer. To maintain a steady pace of learning for years, cultivate the habit of learning a little bit every week. Learning technical skills for a promising AI career involves learning foundational technical skills, working on projects, and finding a job supported by being part of a community."

In [11]:

# source_nodes   returns a List  of *NodeWithScore* object  that
# contain the most top_k chunks relevants to the query
result.source_nodes

# response.metadata  # also contains data relevant to the context used

[NodeWithScore(node=TextNode(id_='f16683cc-d3c5-4dd1-9897-a3166a2402ef', embedding=None, metadata={'window': 'Fogg explains that the best way to build a new habit is to start small \nand succeed, rather than start  too big and fail.  For example, rather than trying to \nexercise for 30 minutes a day, he recommends aspiring to do just one push-up, and \ndoing it consistently.\n This approach may be helpful to those of you who want to spend more time studying. \n If you start by holding yourself accountable for watching, say, 10 seconds of an \neducational video every day — and you do so consistently — the habit of studying daily \nwill grow naturally.  Even if you learn nothing in that 10 seconds, you’re establishing the \nhabit of studying a little every day.  On some days, maybe you’ll end up studying for an \nhour or longer.', 'original_text': 'If you start by holding yourself accountable for watching, say, 10 seconds of an \neducational video every day — and you do so consistently —

In [12]:
out_dict = {
    'query': question,
    'response': result.response,
    'documents': [node.metadata['file_name'] for node in result.source_nodes],
    'pages': [node.metadata['page_label'] for node in result.source_nodes]

}
out_dict

{'query': 'how can I start learning on the AI way?',
 'response': "The best way to start learning on the AI way is to start small and succeed. Instead of trying to exercise for 30 minutes a day, aim for doing just one push-up. This approach may be helpful for those who want to spend more time studying. Even if you learn nothing in that 10 seconds, you're building the habit of studying daily. On some days, you may end up studying for an hour or longer. To maintain a steady pace of learning for years, cultivate the habit of learning a little bit every week. Learning technical skills for a promising AI career involves learning foundational technical skills, working on projects, and finding a job supported by being part of a community.",
 'documents': ['eBook-How-to-Build-a-Career-in-AI.pdf',
  'eBook-How-to-Build-a-Career-in-AI.pdf',
  'eBook-How-to-Build-a-Career-in-AI.pdf'],
 'pages': ['11', '10', '9']}

## View the resources used

In [1]:
import psutil
import GPUtil
import time


def performance_metrics(function):
    def wrapper(*args, **kwargs):
        gpus = GPUtil.getGPUs()
        ram_info = psutil.virtual_memory()
        init_ram = ram_info.available / (1024 ** 3)
        init_gpu = gpus[0].memoryFree
        start_time = time.time()

        out = function(*args, **kwargs)

        gpus = GPUtil.getGPUs()
        ram_info = psutil.virtual_memory()
        print(
            f'Used RAM: {round(init_ram - ram_info.available / (1024 ** 3),3)} GB')
        print(f'Used GPU: {round(init_gpu - gpus[0].memoryFree,3)} MB')
        print(f'Consumed time {time.time() - start_time} seconds')

        return out
    return wrapper

## Put all together

In [2]:
import os
# Hugging Face Tokenizers Warning: This warning is related to the parallelism feature in 
# Hugging Face’s tokenizers library. When a process that has already used parallelism
# gets forked, it can lead to deadlocks, which is why parallelism is disabled.

# Avoid using tokenizers before the fork if possible.
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [3]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.embeddings import resolve_embed_model
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index import VectorStoreIndex, StorageContext, ServiceContext
from llama_index import load_index_from_storage


@performance_metrics
def build_index(files, embed_model='local:BAAI/bge-large-en-v1.5', model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', persist_dir='LLM_TRAIN/sentence_index'):

    documents = SimpleDirectoryReader(input_files=files).load_data()

    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        include_prev_next_rel=True,
        original_text_metadata_key="original_text",
    )
    embed_model = resolve_embed_model(embed_model)
    llm = HuggingFaceLLM(
        model_name=model,
        tokenizer_name=model,
        query_wrapper_prompt=PromptTemplate(
            "<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
        context_window=2048,  # 4096
        max_new_tokens=256,  # 512
        model_kwargs={'trust_remote_code': True},
        generate_kwargs={"temperature": 0.7, "do_sample": True},
        device_map="auto",
    )

    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )

    if not os.path.exists(persist_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )

        sentence_index.storage_context.persist(persist_dir=persist_dir)

    else:
        # print('Loading index from disk')
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=persist_dir),
            service_context=sentence_context)

    return sentence_index


@performance_metrics
def query_index(index, question):

    # This takes a value stored in the metadata and replaces a node text
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

    sentence_window_engine = index.as_query_engine(
        similarity_top_k=3,  # fetch the most similarity
        node_postprocessors=[postproc])

    result = sentence_window_engine.query(question)
    return {
        'query': question,
        'response': result.response,
        'documents': [node.metadata['file_name'] for node in result.source_nodes],
        'pages': [node.metadata['page_label'] for node in result.source_nodes]
    }

## MODEL: TinyLlama/TinyLlama-1.1B-Chat-v1.0

In [4]:
sentence_index = build_index(
    files=["LLM_TRAIN/eBook-How-to-Build-a-Career-in-AI.pdf"],
    embed_model="local:BAAI/bge-large-en-v1.5",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    persist_dir="LLM_TRAIN/sentence_index",
)

Used RAM: 2.276 GB
Used GPU: 5623.0 MB
Consumed time 7.663482666015625 seconds


In [5]:
dict_response = query_index(sentence_index,
      "What are the keys to building a career in AI?")
dict_response

Used RAM: 0.047 GB
Used GPU: 254.0 MB
Consumed time 3.1760852336883545 seconds


{'query': 'What are the keys to building a career in AI?',
 'response': 'The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job. These steps stack on top of each other and are part of a broader process of gaining experience, building a portfolio, and finding a job. Chapters with focused topics on learning foundational technical skills include learning about AI concepts and methodologies, building a portfolio, and creating impact. The key to building a career in AI is to focus on learning foundational skills, working on projects, and finding a job, all of which are supported by being part of a community.',
 'documents': ['eBook-How-to-Build-a-Career-in-AI.pdf',
  'eBook-How-to-Build-a-Career-in-AI.pdf',
  'eBook-How-to-Build-a-Career-in-AI.pdf'],
 'pages': ['9', '6', '35']}